# From TTS data to market level characteristics 
- update: June 25, 2020

## input: Import ES labels and coordinates

In [40]:
parameter='90_100_2two_step'

In [1]:
import pandas as pd 
cluster_labels=pd.read_csv('../2_pipeline/es_labeled90_100_2two_step.csv')

In [2]:
cluster_labels.head()

,Unnamed: 0,labels,lat,long
0,0,0,45.569080,-122.946701
1,1,14,42.503670,-83.781059
2,2,4,34.004478,-117.684700
3,3,6,33.610741,-111.891472
4,4,30,29.985395,-95.346214


In [3]:
cluster_labels.describe()

,Unnamed: 0,labels,lat,long
count,463.000000,463.000000,463.000000,463.000000
mean,136.354212,16.699784,37.798171,-93.395793
std,93.665795,13.202655,4.895085,19.449196
min,0.000000,-1.000000,25.485147,-122.987328
25%,60.000000,4.000000,34.068075,-117.088020
50%,118.000000,17.000000,38.952515,-84.275970
75%,208.500000,30.500000,41.626019,-74.911308
max,416.000000,36.000000,48.796490,-69.747612


## Intermediate step - attach centroid to every ES installer 

### Helper function: get_coords 

In [4]:
def get_coords( dataframe ):
    #df=dataframe[['installerid','lat','long']]
    df=dataframe[['lat','long']]
    df=df.dropna()
    coords=df.as_matrix(columns=['lat','long'])
    return coords;



### Helper function: generate the centroid for each ES market 

In [5]:
from shapely.geometry import MultiPoint
def get_centroid(cluster):
    centroid = (MultiPoint(cluster).centroid.y, MultiPoint(cluster).centroid.x)
    return tuple(centroid)

### add centroid to ES market data 

In [7]:
import numpy as np
cluster_labels['mkt_centroid_la']=np.nan
cluster_labels['mkt_centroid_long']=np.nan

In [8]:
cluster_labels.head()

,Unnamed: 0,labels,lat,long,mkt_centroid_la,mkt_centroid_long
0,0,0,45.569080,-122.946701,NaN,NaN
1,1,14,42.503670,-83.781059,NaN,NaN
2,2,4,34.004478,-117.684700,NaN,NaN
3,3,6,33.610741,-111.891472,NaN,NaN
4,4,30,29.985395,-95.346214,NaN,NaN


### by labels( market index) compute the centroid of each cluster 

In [9]:
for index,row in cluster_labels.iterrows():
    label=row.labels
    centroid=get_centroid(get_coords(cluster_labels[cluster_labels['labels']==label]))
    cluster_labels['mkt_centroid_long'].iloc[index]=centroid[0]
    cluster_labels['mkt_centroid_la'].iloc[index]=centroid[1]
    

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
C:\Anaconda\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [10]:
cluster_labels.head()

,Unnamed: 0,labels,lat,long,mkt_centroid_la,mkt_centroid_long
0,0,0,45.569080,-122.946701,45.437519,-122.771051
1,1,14,42.503670,-83.781059,42.403030,-83.678697
2,2,4,34.004478,-117.684700,33.721546,-117.710008
3,3,6,33.610741,-111.891472,33.512993,-111.903642
4,4,30,29.985395,-95.346214,29.759703,-95.511000


In [12]:
es_market_centroids=cluster_labels[['labels','mkt_centroid_la','mkt_centroid_long']].drop_duplicates()

In [13]:
es_market_centroids.head()

,labels,mkt_centroid_la,mkt_centroid_long
0,0,45.437519,-122.771051
1,14,42.403030,-83.678697
2,4,33.721546,-117.710008
3,6,33.512993,-111.903642
4,30,29.759703,-95.511000


## Helper Functions

### Helper function: Haversine distance

In [14]:
from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):

      R = 3959.87433 # this is in miles.  For Earth radius in kilometers use 6372.8 km

      dLat = radians(lat2 - lat1)
      dLon = radians(lon2 - lon1)
      lat1 = radians(lat1)
      lat2 = radians(lat2)

      a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
      c = 2*asin(sqrt(a))

      return R * c

### Find TTS installers within each ES market 

In [15]:
def find_tts_from_centroid(centroid_la,centroid_long,tts,threshold_r):
    temp_tts['dis_to_centroid_pt']=temp_tts.apply(lambda x:haversine(x['lat'],x['long'],centroid_la,centroid_long),axis=1)
   
    return set(temp_tts[temp_tts['dis_to_centroid_pt']<threshold_r].installerid)

In [16]:
def find_tts_monthly(ttsids,ttsmonthly):
    if(len(ttsids)>0):
        return ttsmonthly.loc[ttsmonthly['installername2'].isin(ttsids)]
    else :
        return

## input: import US Zip code data 

In [17]:
zipcode_coords_url= "../0_data/RawData_geospatial//uszip_latlong.dta" 

In [18]:
zipcode_coords=pd.read_stata(zipcode_coords_url)

In [19]:
zipcode_coords.head()

,zipcode_str,lat,lng
0,00601,18.180555,-66.749962
1,00602,18.361944,-67.175598
2,00603,18.455183,-67.119888
3,00606,18.158327,-66.932930
4,00610,18.295366,-67.125137


In [21]:
zipcode_coords['zipcode']=zipcode_coords['zipcode_str'].astype(int)

In [22]:
zipcode_coords.describe()

,lat,lng,zipcode
count,33144.000000,33144.000000,33144.000000
mean,38.819046,-90.926079,49668.659033
std,5.397584,15.788619,27585.182370
min,-14.223174,-176.629547,601.000000
25%,35.390397,-97.234253,26622.500000
50%,39.493353,-88.194725,49741.500000
75%,42.120234,-80.220591,72131.250000
max,71.251678,145.754395,99929.000000


## input:  import zipcode level revenue data 

In [27]:
pwd

'C:\\Users\\Herbie Huang\\Google Drive\\Current_SolarResearch\\working_june2020\\solar_project\\1_code'

In [32]:
tts_monthlyrev_byzip_url="../0_data/tts_monthlyrev_byzip.dta"

In [33]:
tts_monthlyrev_byzip=pd.read_stata(tts_monthlyrev_byzip_url)

In [34]:
tts_monthlyrev_byzip.describe()

,zipcode,year,month,zipcode_total_rev
count,140453.000000,140453.000000,140453.000000,1.404530e+05
mean,55241.661773,2015.050171,6.719073,1.850377e+05
std,41340.762148,1.689698,3.437629,1.286470e+07
min,501.000000,2012.000000,1.000000,1.000000e-02
25%,10920.000000,2013.000000,4.000000,3.270000e+04
50%,85304.000000,2015.000000,7.000000,6.782950e+04
75%,92675.000000,2016.000000,10.000000,1.639100e+05
max,97918.000000,2017.000000,12.000000,4.807869e+09


## helper function : find the closest zipcodes 

In [35]:
def find_zips_from_centroid(centroid_la,centroid_long,zipcode_coords,threshold_r):
    temp=zipcode_coords
    temp['dis_to_centroid_pt']=temp.apply(lambda x:haversine(x['lat'],x['lng'],centroid_la,centroid_long),axis=1)
    return set(temp[temp['dis_to_centroid_pt']<threshold_r].zipcode)

In [36]:
es_market_centroids.head()

,labels,mkt_centroid_la,mkt_centroid_long
0,0,45.437519,-122.771051
1,14,42.403030,-83.678697
2,4,33.721546,-117.710008
3,6,33.512993,-111.903642
4,30,29.759703,-95.511000


In [37]:
def find_tts_monthly(zipcodes,ttsmonthly_byzip):
    if(len(zipcodes)>0):
        return ttsmonthly_byzip.loc[ttsmonthly_byzip['zipcode'].isin(zipcodes)]
    else :
        return

# Use tts_monthlyrev_byzip to create zipcode level total revenue data that is linked to market label

In [38]:
market_condition=pd.DataFrame(columns=tts_monthlyrev_byzip.columns)
for index,row in es_market_centroids.iterrows():
    market_label=row['labels']
    centroid_la=row['mkt_centroid_la']
    centroid_long=row['mkt_centroid_long']
    tts_search_radius=100
    nearby_zips=find_zips_from_centroid(centroid_la,centroid_long,zipcode_coords,tts_search_radius)
    df_tts=find_tts_monthly(nearby_zips,tts_monthlyrev_byzip)
    try: 
        df_tts.loc[:,'market']=market_label
        market_condition=market_condition.append(df_tts,ignore_index=True)
    except:
        print('market '+str(market_label)+' has an exception...probably a lack of TTS installers')
    

C:\Anaconda\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Anaconda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Anaconda\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and

market 14.0 has an exception...probably a lack of TTS installers
market 18.0 has an exception...probably a lack of TTS installers
market -1.0 has an exception...probably a lack of TTS installers
market 9.0 has an exception...probably a lack of TTS installers
market 32.0 has an exception...probably a lack of TTS installers
market 21.0 has an exception...probably a lack of TTS installers
market 1.0 has an exception...probably a lack of TTS installers
market 8.0 has an exception...probably a lack of TTS installers
market 24.0 has an exception...probably a lack of TTS installers
market 16.0 has an exception...probably a lack of TTS installers
market 26.0 has an exception...probably a lack of TTS installers
market 27.0 has an exception...probably a lack of TTS installers
market 23.0 has an exception...probably a lack of TTS installers
market 22.0 has an exception...probably a lack of TTS installers
market 19.0 has an exception...probably a lack of TTS installers


In [42]:
parameter

'90_100_2two_step'

## Output: market condition variable 

In [53]:
market_condition.tail()

,market,month,year,zipcode,zipcode_total_rev
177004,36.0,12.0,2017.0,6712,20793.630859
177005,36.0,12.0,2017.0,6716,26982.980469
177006,36.0,12.0,2017.0,6779,54088.820312
177007,36.0,12.0,2017.0,6787,25656.000000
177008,36.0,12.0,2017.0,6790,206624.765625


In [43]:
market_condition.to_csv('../2_pipeline/marketconditions'+parameter+'.csv')

## From zip level Market Condition Data to Market Level 
Create data to be directly merged back 

In [44]:
parameter

'90_100_2two_step'

In [45]:
import pandas as pd 
market_conditions_ziplevel=pd.read_csv('../2_pipeline/marketconditions'+parameter+'.csv')

In [46]:
market_conditions_ziplevel.tail()

,Unnamed: 0,market,month,year,zipcode,zipcode_total_rev
177004,177004,36.0,12.0,2017.0,6712,20793.63
177005,177005,36.0,12.0,2017.0,6716,26982.98
177006,177006,36.0,12.0,2017.0,6779,54088.82
177007,177007,36.0,12.0,2017.0,6787,25656.00
177008,177008,36.0,12.0,2017.0,6790,206624.77


In [47]:
market_conditions_ziplevel=market_conditions_ziplevel.drop(columns=['Unnamed: 0'],axis=1)

In [48]:
market_conditions_ziplevel[(market_conditions_ziplevel['market']==34)&(market_conditions_ziplevel['month']==12)&(market_conditions_ziplevel['year']==2016)].zipcode_total_rev.sum()

30351518.7

### Sum up revenue per market 

In [49]:
mkt_condition_mktlevel=market_conditions_ziplevel.groupby(["market", "month","year"]).zipcode_total_rev.sum().reset_index() 

In [50]:
mkt_condition_mktlevel.tail()

,market,month,year,zipcode_total_rev
1333,36.0,12.0,2013.0,2.310547e+07
1334,36.0,12.0,2014.0,7.154613e+07
1335,36.0,12.0,2015.0,1.017081e+08
1336,36.0,12.0,2016.0,6.386177e+07
1337,36.0,12.0,2017.0,3.209811e+07


## Output: market condition variable organized as market-month level 

In [51]:
parameters='90_100_2two_step'

In [ ]:
mkt_condition_mktlevel.to_csv('../2_pipeline/tts_mktcondidtions'+parameters+'.csv')